# Recall (Last Class)
* OS Components
* Role of OS

## What happens when CPU is turned on/reset
* Starts an execution cycle, fetches an instruction
* For x86, PC/EIP is set to `0xFFFFFFF0` for the first instruction
    * This is 16 bytes from the end of the valid memory region
    * Note that you usually won't even have this much memory
    * This is a small chunk of code, so this is usually just a `JMP` instruction

The standard is to jump to BIOS at `0xF000`
* BIOS usually appeared in ROM addressed right after the last RAM address
* We now have so much RAM that we go over where the BIOS is
* For legacy compatibility, we leave the BIOS there

## Basic computer model
1. Fetch instruction
2. Parse instruction
3. Execute
4. Store result

## BIOS
* All that the BIOS needs to do is read sector 0 from a device (eg. disk, flash, floppy, etc)
* The first block is copied into RAM and then the program counter jumps into it
    * This block is typically a bootloader, loads something bigger
    * This bigger code typically knows how to handle different disk formats (eg. NTFS, ext4, HFS+) and starts loading the kernel
* This bootstrapping process is to allow the startup for a processor to be generic

# Post-Boot

The memory layout for a simple system after bootloader transfer control to kernel:

```
0x0
----------------------------------
| OS | User Program resides here |
----------------------------------
```
Since we want to run more than one process at a time, we want to be able to multiplex main memory

### Multi-Process Model
* Share main memory across multiple processes
* Allow execution of other processes while a process is waiting for some signal

## Step 1: Basic Memory Management
* Write `kmalloc` and `kfree` to allow the system to grab memory areas

```
State of memory:
---------------------------------------
| Kernel |   free   | HOLE |   free   |
---------------------------------------
```

The hole is problematic.

### Incremental Design at a Memory Manager
1. Alpha
    * Ignore the hole for now
    * Ignore free
    * Keep a pointer to the spot where free memory starts. On kmalloc, increment pointer and return the address
    * You'll run out of memory eventually, but at least your processes can have memory!
2. Adding Free
    * Ideas: Use a linked list for allocated and freed spaces
    * This fragmentates our memory
    * We also need memory to store the list. We can't call kmalloc to create memory for the list
    * So, instead of using a dynamic linked list, maybe we should use an array
    * However, arrays may not be optimal for dynamic allocation. Small allocations will deplete your table. Large allocations will keep your table sparse.